1.Predict whata n existing customer wants to watch


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns

!pip install surprise
from surprise import Reader, Dataset, SVD, evaluate
sns.set_style("darkgrid")



     |████████████████████████████████| 3.3MB 4.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
import sys
!{sys.executable} -m pip install scikit-surprise



In [0]:
import sys
!{sys.executable} -m pip install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.1.1)


In [0]:
df1 = pd.read_csv('/content/drive/My Drive/Project Group 01/ml-latest/ratings.csv')


In [0]:
df1.head().transpose()

,0,1,2,3,4
userId,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
movieId,3.070000e+02,4.810000e+02,1.091000e+03,1.257000e+03,1.449000e+03
rating,3.500000e+00,3.500000e+00,1.500000e+00,4.500000e+00,4.500000e+00
timestamp,1.256677e+09,1.256677e+09,1.256677e+09,1.256677e+09,1.256677e+09


In [0]:
df1['Rating'] = df1['rating'].astype(float)
print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')
print(df1.iloc[::100000, :])


Dataset 1 shape: (27753444, 5)
-Dataset examples-
          userId  movieId  rating   timestamp  Rating
0              1      307     3.5  1256677221     3.5
100000      1041     1498     4.0   939449322     4.0
200000      2025      170     2.0  1452364479     2.0
300000      2929      160     3.5  1111751954     3.5
400000      4084      590     3.0  1171658068     3.0
500000      5081     2353     3.5  1230594013     3.5
600000      6085      480     5.0  1351459814     5.0
700000      7027      575     5.0   992894989     5.0
800000      8051    78122     4.0  1516866889     4.0
900000      9011     1374     4.0  1134576825     4.0
1000000    10073     1485     3.5  1535184421     3.5
1100000    11007     8154     2.0  1444944010     2.0
1200000    12077   104879     3.5  1519748010     3.5
1300000    13094        7     3.0   854203278     3.0
1400000    14144     2731     4.5  1450471561     4.5
1500000    15224     8528     3.5  1342292904     3.5
1600000    16261     1283     5.

In [0]:
df = df1

df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::50000, :])
'sort=True'


Full dataset shape: (27753444, 5)
-Dataset examples-
          userId  movieId  rating   timestamp  Rating
0              1      307     3.5  1256677221     3.5
50000        491     2445     3.0   974953235     3.0
100000      1041     1498     4.0   939449322     4.0
150000      1479     6874     4.0  1112134694     4.0
200000      2025      170     2.0  1452364479     2.0
250000      2463    71379     3.0  1484599936     3.0
300000      2929      160     3.5  1111751954     3.5
350000      3552    59315     4.0  1513737430     4.0
400000      4084      590     3.0  1171658068     3.0
450000      4605    71433     2.0  1450801410     2.0
500000      5081     2353     3.5  1230594013     3.5
550000      5584   135133     4.0  1461277444     4.0
600000      6085      480     5.0  1351459814     5.0
650000      6524      260     4.0  1016156637     4.0
700000      7027      575     5.0   992894989     5.0
750000      7633     2117     3.5  1083301636     3.5
800000      8051    78122    

'sort=True'

In [0]:
f = ['count','mean']

df_movie_summary = df.groupby('movieId')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.8),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df.groupby('userId')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.8),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

Movie minimum times of review: 91.0
Customer minimum times of review: 122.0


In [0]:
df=df.drop(['rating','timestamp'],axis=1)

In [0]:
print('Original Shape: {}'.format(df.shape))
df = df[~df['movieId'].isin(drop_movie_list)]
df = df[~df['userId'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])

Original Shape: (27753444, 3)
After Trim Shape: (19868906, 3)
-Data Examples-
          userId  movieId  Rating
42             4        1     4.0
6978803    71695     1163     5.0
13975464  143056      337     3.0
20954410  213991      942     5.0


In [0]:
'''df_p = pd.pivot_table(df,values='Rating',index='userId',columns='movieId')

print(df_p.shape)'''

"df_p = pd.pivot_table(df,values='Rating',index='userId',columns='movieId')\n\nprint(df_p.shape)"

In [0]:
df_title = pd.read_csv('/content/drive/My Drive/Project Group 01/ml-latest/movies.csv', encoding = "ISO-8859-1",)
df_title.set_index('movieId', inplace = True)
print (df_title.head(10))

                                      title                                       genres
movieId                                                                                 
1                          Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
2                            Jumanji (1995)                   Adventure|Children|Fantasy
3                   Grumpier Old Men (1995)                               Comedy|Romance
4                  Waiting to Exhale (1995)                         Comedy|Drama|Romance
5        Father of the Bride Part II (1995)                                       Comedy
6                               Heat (1995)                        Action|Crime|Thriller
7                            Sabrina (1995)                               Comedy|Romance
8                       Tom and Huck (1995)                           Adventure|Children
9                       Sudden Death (1995)                                       Action
10                   

In [0]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['userId', 'movieId', 'Rating']], reader)
data.split(n_folds=3)

svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/lib/python3.6/dist-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


In [0]:
df_1014 = df[(df['userId'] == 1014) & (df['Rating'] == 5)]
df_1014 = df_1014.set_index('movieId')
df_1014 = df_1014.join(df_title)['title']
print(df_1014)

movieId
17                           Sense and Sensibility (1995)
34                                            Babe (1995)
36                                Dead Man Walking (1995)
50                             Usual Suspects, The (1995)
121                       Boys of St. Vincent, The (1992)
176                             Living in Oblivion (1995)
260             Star Wars: Episode IV - A New Hope (1977)
372                                  Reality Bites (1994)
376                                River Wild, The (1994)
377                                          Speed (1994)
480                                  Jurassic Park (1993)
527                               Schindler's List (1993)
581                          Celluloid Closet, The (1995)
589                     Terminator 2: Judgment Day (1991)
593                      Silence of the Lambs, The (1991)
595                           Beauty and the Beast (1991)
608                                          Fargo (1996)
838   

In [0]:
user_1014 = df_title.copy()
user_1014 = user_1014.reset_index()
#user_17 = user_17[~user_17['movieId'].isin(drop_movie_list)]

# getting full dataset
data = Dataset.load_from_df(df[['userId', 'movieId', 'Rating']], reader)

trainset = data.build_full_trainset()
svd.fit(trainset)

user_1014['Estimate_Score'] = user_1014['movieId'].apply(lambda x: svd.predict(1014, x).est)

user_1014 = user_1014.drop('movieId', axis = 1)

user_1014 = user_1014.sort_values('Estimate_Score', ascending=False)
print(user_1014.head(10))

                                                  title  \
908                        To Kill a Mockingbird (1962)   
836                   E.T. the Extra-Terrestrial (1982)   
510                    Silence of the Lambs, The (1991)   
1048  Paradise Lost: The Child Murders at Robin Hood...   
701                            Wizard of Oz, The (1939)   
920                                       Psycho (1960)   
7829                              Civil War, The (1990)   
1472                               Exorcist, The (1973)   
1067                                        Jaws (1975)   
2110                          Christmas Story, A (1983)   

                                  genres  Estimate_Score  
908                                Drama             5.0  
836                Children|Drama|Sci-Fi             5.0  
510                Crime|Horror|Thriller             5.0  
1048                         Documentary             5.0  
701   Adventure|Children|Fantasy|Musical             5.

Predict movies for new users

Find if a movie will be a hit or not